# Tutorial 3: データ構造とアルゴリズム

8. [](#sortable-dataclass)
9. [](#heap)

アルゴリズム論の3.7節「ヒープ」も適宜参照してください [^note1]．

[^note1]: ただし，本講義ではヒープの実装そのものは扱わない．
    一方で，ヒープがどういう構造と特徴を持っているか，ということは理解していることを望んでいる．


In [1]:
%%html
<style>
    div.jp-CodeMirrorEditor span.ͼ11 { font-style: normal; } /* Comment (Jupyter 4.x) */
    div.jp-MarkdownCell h2 { border-bottom: solid 2px #aaf; padding-bottom: 2px; }
</style>

In [2]:
from IPython.display import display

(sortable-dataclass)=
## ソート可能なデータクラス

前回利用した dataclass，つまりある種の構造体は，直接その大小を比較することはできません．  
この事実自体は，C言語の構造体で <u>[同じ問題に直面](https://edu2024.sp.cs.okayama-u.ac.jp/eop/p2/p/practice08-hint.html#id13)</u> したことがあるはず．  
そして，「対象となるデータの大小関係を定められるならば，任意のデータの並べ替えが可能」という事実を学んだはずです [^ext3]．

[^ext3]: 3年次編入の人は，すでに同等の概念を学んでいることを期待します・・・

Pythonのデータ構造の大半はクラスとして定義されており，
その中でも大小比較が可能なクラスは，`__lt__()`を実装することになっています．  
例えば，年月日を意味する `Date` クラスであれば，以下に示すような `__lt__(a, b)` 関数によって，
`a > b` か否かを判定できるはずです．

In [3]:
from dataclasses import dataclass

@dataclass    # <-- class定義の前にアットマーク付きのデコレータが必要です．やや特殊．．．
class Date:
    y: int
    m: int
    d: int

    def __lt__(a, b):  # Less than function: "A is less than B".
        if a.y != b.y:
            return a.y < b.y
        if a.m != b.m:
            return a.m < b.m
        return a.d < b.d


実は上記のような `Date` クラスを定義しておくと，`Date` を複数持つリストに対してソートが可能になります[^ltgt]．  
内部でどういう動作をしているかは，プログラミング演習2でソートを実装した当時のことを思い出してください．

[^ltgt]: やや繰り返しになるのですが，ソートできることはさほど重要ではないです．大小関係を与えられるという点が，とっっっっっても重要です．

In [4]:
d1 = Date(2012, 5, 1)
d2 = Date(2011, 5, 2)
d3 = Date(2018, 3, 23)

dates = [d1, d2, d3]
display(f'Before sort: {dates}')

dates = sorted(dates)  # default は昇順 (Ascendant)
display(f'After sort1: {dates}')

dates = sorted(dates, reverse=True)  # reverse=True は降順 (Descendant)
display(f'After sort2: {dates}')


'Before sort: [Date(y=2012, m=5, d=1), Date(y=2011, m=5, d=2), Date(y=2018, m=3, d=23)]'

'After sort1: [Date(y=2011, m=5, d=2), Date(y=2012, m=5, d=1), Date(y=2018, m=3, d=23)]'

'After sort2: [Date(y=2018, m=3, d=23), Date(y=2012, m=5, d=1), Date(y=2011, m=5, d=2)]'

```{exercise}
AI-1では State データクラスを作りました．cost に基づいて大小比較ができるように拡張してみましょう．

以下のセルを利用して書いてください．
```

In [18]:
@dataclass
class State:
    """
    Parameters
    ----------
    id: int
        ロボットが存在する場所．探索したいグラフのノード番号で指定する．
        つまり，Adjacency Matrix 等のグラフ表現における番号と相互参照できるものとする．

    cost: float
        探索過程におけるノードのコスト g(n)

    parent_id: int
        現在位置に至る1つ前の時点で滞在していた場所．ノード番号で指定する．
        値が負のとき，親ノードは存在しない（すなわち，root note）とする．

    Note
    ----
    - cost に基づいてソート可能である
    """
    
    id: int
    cost: float
    parent_id: int

    def __lt__(a, b):
        if a.cost != b.cost:
            return a.cost < b.cost

In [19]:
# Test cell
x1 = State(0, cost=10, parent_id=-1)
x2 = State(1, cost=4, parent_id=-1)
x3 = State(2, cost=11, parent_id=-1)
x4 = State(3, cost=7, parent_id=-1)

l = [x1, x2, x3, x4]

print(l)
print(sorted(l, reverse=False))
print(sorted(l, reverse=True))
    # 昇順ソート，降順ソートがそれぞれ実現できていることを確かめてください．

[State(id=0, cost=10, parent_id=-1), State(id=1, cost=4, parent_id=-1), State(id=2, cost=11, parent_id=-1), State(id=3, cost=7, parent_id=-1)]
[State(id=1, cost=4, parent_id=-1), State(id=3, cost=7, parent_id=-1), State(id=0, cost=10, parent_id=-1), State(id=2, cost=11, parent_id=-1)]
[State(id=2, cost=11, parent_id=-1), State(id=0, cost=10, parent_id=-1), State(id=3, cost=7, parent_id=-1), State(id=1, cost=4, parent_id=-1)]


(heap)=
## ヒープ (データ構造)

AI-2では，より高度な探索を実現するため，ヒープ構造を使います [^heapsort]．  
ヒープ構造とは「最小値のデータを取り出しやすい構造」です [^general_heap]．

[^heapsort]: ヒープソートではありません．ヒープ構造はソートすることが主目的ではありません．
[^general_heap]: 参考書アルゴリズム論（ならびに，一般的な同分野の教科書）では，「最大値のデータを取り出しやすい構造」として紹介されます．
    ここでは，Pythonの `heapq` 実装に合わせ，「最大」ではなく「最小」として理解を進めてください．

様々なアルゴリズムを実装する中で，「リストの最もｘｘなデータ」を取り出すことはよくあります．  
実際，Stackとは「ｘｘな＝最後に入れられた」ですし，Queueとは「ｘｘな＝最初に入れられた」です．

今回は，「ｘｘな＝Stateのcostが小さい」を考えます．  
このようなデータ構造は cost を priority (優先度) とみなしたとき， 優先度付きキュー (Priority Queue) とも呼びます．  
このPriority Queue を実現するには，ヒープ構造がうってつけ，ということです．

pythonでは `heapq` モジュールにヒープデータ構造として扱うための各種関数が定義されています．

- 参考：heapq --- ヒープキューアルゴリズム https://docs.python.org/ja/3.12/library/heapq.html

以下に，deque を使った Queue との対比を載せます．少し癖のある関数群ですので注意して読み進めてください．

In [32]:
# 以下の state1, ..., state4 を states_queue に入れることを考えてみよう．
state1 = State(0, cost=10, parent_id=-1)
state2 = State(1, cost= 8, parent_id=-1)
state3 = State(5, cost=12, parent_id=-1)
state4 = State(5, cost= 9, parent_id=-1)  # <-- state3と同じIDだが，costがstate3より低い
state5 = State(1, cost=11, parent_id=-1)  # <-- state2と同じIDだが，costがstate2より高い
state6 = State(1, cost= 7, parent_id=-1)
state7 = State(1, cost= 6, parent_id=-1)

### Simple Queue

まずは基本を思い出してもらうために，Tutoral 2 と同じ Queue を示します．simple queue はヒープではありません．念のため．．．

In [33]:
from collections import deque
##
## (Simple) Queue
##

# ---------------------------------------------------------
# initialize
print('----- initialize')
state_queue = deque()
print(state_queue)

----- initialize
deque([])


In [34]:
# ---------------------------------------------------------
# enqueue
#   - forループの中で，1要素ずつ enqueue しています
print('\n----- enqueue')
for n in [state1, state2, state3, state4, state5, state6, state7]:
    state_queue.append(n)
    print(state_queue)

#DEBUG
print('STATE_QUEUE:')
display(state_queue)


----- enqueue
deque([State(id=0, cost=10, parent_id=-1)])
deque([State(id=0, cost=10, parent_id=-1), State(id=1, cost=8, parent_id=-1)])
deque([State(id=0, cost=10, parent_id=-1), State(id=1, cost=8, parent_id=-1), State(id=5, cost=12, parent_id=-1)])
deque([State(id=0, cost=10, parent_id=-1), State(id=1, cost=8, parent_id=-1), State(id=5, cost=12, parent_id=-1), State(id=5, cost=9, parent_id=-1)])
deque([State(id=0, cost=10, parent_id=-1), State(id=1, cost=8, parent_id=-1), State(id=5, cost=12, parent_id=-1), State(id=5, cost=9, parent_id=-1), State(id=1, cost=11, parent_id=-1)])
deque([State(id=0, cost=10, parent_id=-1), State(id=1, cost=8, parent_id=-1), State(id=5, cost=12, parent_id=-1), State(id=5, cost=9, parent_id=-1), State(id=1, cost=11, parent_id=-1), State(id=1, cost=7, parent_id=-1)])
deque([State(id=0, cost=10, parent_id=-1), State(id=1, cost=8, parent_id=-1), State(id=5, cost=12, parent_id=-1), State(id=5, cost=9, parent_id=-1), State(id=1, cost=11, parent_id=-1), State

deque([State(id=0, cost=10, parent_id=-1),
       State(id=1, cost=8, parent_id=-1),
       State(id=5, cost=12, parent_id=-1),
       State(id=5, cost=9, parent_id=-1),
       State(id=1, cost=11, parent_id=-1),
       State(id=1, cost=7, parent_id=-1),
       State(id=1, cost=6, parent_id=-1)])

In [35]:
# ---------------------------------------------------------
# dequeue
print('\n----- dequeue (1)')
a = state_queue.popleft()
print(a)

print('\n----- dequeue (2)')
a = state_queue.popleft()
print(a)

# DEBUG
print('STATE_QUEUE:')
display(state_queue)


----- dequeue (1)
State(id=0, cost=10, parent_id=-1)

----- dequeue (2)
State(id=1, cost=8, parent_id=-1)
STATE_QUEUE:


deque([State(id=5, cost=12, parent_id=-1),
       State(id=5, cost=9, parent_id=-1),
       State(id=1, cost=11, parent_id=-1),
       State(id=1, cost=7, parent_id=-1),
       State(id=1, cost=6, parent_id=-1)])

### Priority Queue (heap)

道中で `state_queue` を表示しています．これは heap のコンテナとなるリストです．

以下の質問に答えられるように，読み解いてください．

(pq-question)=
- `state_queue[0]` はどういう性質のデータか？（例えば，「最初に挿入されたデータである」のように，データ構造として，先頭(`[0]`)に求められる性質を考えよ，という意味）
- `state_queue` そのものは，大小関係に基づくソート順に並んで配置されているか？（つまり，`sorted(state_queue)` と同じ並びになるか？という意味）

> WWWページとして閲覧中の方への注意：以下の例は，上の方の State の実装にかかる Exercise が完成していないと，想定した表示にはなりません．

繰り返しますが，ヒープソートとヒープ構造を混同しないようにしてください．
必要に応じて，[公式リファレンス](https://docs.python.org/ja/3.12/library/heapq.html) も参考にしましょう．

In [41]:
import heapq
##
## Priority Queue (Heap data structure)
##

# ---------------------------------------------------------
# initialize
print('----- initialize')
state_queue = []
heapq.heapify(state_queue)
print(state_queue)
    # - heqpq.heap**** は deque が使えません．通常の list を使います．
    # - 任意のリストは heapify を経ることで heap として扱えるようになります．
    #   - ただし，ヒープ構造の特徴からして，1つ以下の要素しかないなら heapify は不要です．
    #     （あくまで，初期化の例として，heapify() の利用例を示しています．）

----- initialize
[]


In [42]:
# ---------------------------------------------------------
# enqueue
#   - forループの中で，1要素ずつ enqueue しています
print('\n----- enqueue')
for n in [state1, state2, state3, state4, state5, state6, state7]:
    heapq.heappush(state_queue, n)
    print(state_queue)
        # - heqpq.heappush(_list_, _elem_) を使って，データを挿入します．
        #   - ここで，state_queue.append(n) をしてしまうと，
        #     ヒープ構造が崩れてしまうので，絶対にやってはいけません．
        # - 関数の使い方が今までのポリシーと異なるので注意してください．
        #   - state_queue.heappush(n) ではなく，
        #     heapq.heappush(state_queue, n) です．

#DEBUG
print('STATE_QUEUE:')
display(state_queue)


----- enqueue
[State(id=0, cost=10, parent_id=-1)]
[State(id=1, cost=8, parent_id=-1), State(id=0, cost=10, parent_id=-1)]
[State(id=1, cost=8, parent_id=-1), State(id=0, cost=10, parent_id=-1), State(id=5, cost=12, parent_id=-1)]
[State(id=1, cost=8, parent_id=-1), State(id=5, cost=9, parent_id=-1), State(id=5, cost=12, parent_id=-1), State(id=0, cost=10, parent_id=-1)]
[State(id=1, cost=8, parent_id=-1), State(id=5, cost=9, parent_id=-1), State(id=5, cost=12, parent_id=-1), State(id=0, cost=10, parent_id=-1), State(id=1, cost=11, parent_id=-1)]
[State(id=1, cost=7, parent_id=-1), State(id=5, cost=9, parent_id=-1), State(id=1, cost=8, parent_id=-1), State(id=0, cost=10, parent_id=-1), State(id=1, cost=11, parent_id=-1), State(id=5, cost=12, parent_id=-1)]
[State(id=1, cost=6, parent_id=-1), State(id=5, cost=9, parent_id=-1), State(id=1, cost=7, parent_id=-1), State(id=0, cost=10, parent_id=-1), State(id=1, cost=11, parent_id=-1), State(id=5, cost=12, parent_id=-1), State(id=1, cost=8

[State(id=1, cost=6, parent_id=-1),
 State(id=5, cost=9, parent_id=-1),
 State(id=1, cost=7, parent_id=-1),
 State(id=0, cost=10, parent_id=-1),
 State(id=1, cost=11, parent_id=-1),
 State(id=5, cost=12, parent_id=-1),
 State(id=1, cost=8, parent_id=-1)]

In [43]:
# ---------------------------------------------------------
# dequeue
print('\n----- dequeue (1)')
a = heapq.heappop(state_queue)
print(a)
    # - heqpq.heappop(_list_) を使って，データを取り出します．
    #   - push と同じ注意ですが，state_queue.pop() はおそらく
    #     取り出したい要素にはなりえません．
    #   - deque でもないので，state_queue.popleft() もありません．
    # - 取り出される要素は，リストのある条件を満たした要素です．
    #   - ヒープ構造では，該当要素の配列中の場所は確定しています．
    #   - つまり，state_queue[?] のようにアクセスする意味のある
    #      要素が1つだけあります．（参考：ヒープ構造の性質）

print('\n----- dequeue (2)')
a = heapq.heappop(state_queue)
print(a)

# DEBUG
print('STATE_QUEUE:')
display(state_queue)


----- dequeue (1)
State(id=1, cost=6, parent_id=-1)

----- dequeue (2)
State(id=1, cost=7, parent_id=-1)
STATE_QUEUE:


[State(id=1, cost=8, parent_id=-1),
 State(id=5, cost=9, parent_id=-1),
 State(id=5, cost=12, parent_id=-1),
 State(id=0, cost=10, parent_id=-1),
 State(id=1, cost=11, parent_id=-1)]

```{exercise}
[質問](#pq-question)の答えは考えられましたか？答えをどうぞ．講義中に口頭で答えてもらうかもしれません．
```

> WRITE YOUR ANSWER. This is Markdown cell.

state_queue[0]は配列の中で最小のデータ．ヒープの根である．最小になる根拠はすべての親ノードと子ノードの関係で（親＜子）が成り立つこと．

state_queueそのものは，配列として見てみると大小関係に基づいていない．ヒープの上から幅優先のような形で配列の順序が定まっているから，{0 < 1,2}, {1 < 3,4}, {2 < 5,6},...のような関係．

## 経路探索に適した Prioirty Queue の検討

### Unique-constrained Priority Queue -- ユニーク制約付きの優先度付きキュー

AI-1で見たように，経路探索においては，open_list に様々なデータ構造を適用します．

ただし，open_list に追加する前に (7-1) や (7-2) のような，同一IDの重複を避ける制約[^unique]があります．

[^unique]: [ユニーク制約](https://e-words.jp/w/%E4%B8%80%E6%84%8F%E5%88%B6%E7%B4%84.html)とも呼びます．

最適経路の探索という意義を考えると，

    Priority Queue をベースとしつつ，同一IDが挿入されそうなときはコストが低い状態を優先する（上書きする）

というデータ構造がよさそうです．どうすればよいのでしょうか？例えば，以下のような実装方針が思い浮かびます．

```{mermaid}
flowchart LR
  A(New Datum) --> B{{Exist ID?}}
  B -->|True| C{{"new.cost < exist.cost?"}}
  B -->|False| D[[APPEND]]
  C -->|True| F[[REPLACE]]
  C -->|False| G[[DROP]]
```

- 追加の前に，同一IDがあるかどうか確認する
  - 同一IDがなければ，そのまま追加する (APPEND)
  - 同一IDがあるなら，コストを比較し，低ければ入れ替え (REPLACE)，高ければ捨てる (DROP)
    - ただし，途中のデータを安直に入れ替えたり削除したりすると，heap構造が破壊される
    - そこで，REPLACEとしては，いったん remove と append をしてから，`heapify()` でヒープ木を再構築する

> WWWページとして閲覧中の方への注意：以下の例は，上の方の State の実装にかかる Exercise が完成していないと，想定した表示にはなりません．

In [44]:
import heapq
##
## Unique-constrained Priority Queue (Heap data structure)
##

# ---------------------------------------------------------
# initialize
print('----- initialize')
state_queue = list()
print(state_queue)
    # heqpq は deque が使えません．list()を使いましょう．

----- initialize
[]


In [45]:
# ---------------------------------------------------------
# enqueue
#   - forループの中で，1要素ずつ enqueue しています
print('\n----- enqueue')
for new_state in [state1, state2, state3, state4, state5]:
    _exist_state = [x for x in state_queue if new_state.id == x.id]
    _exist_state = _exist_state[0] if _exist_state else None
    if _exist_state is None:
        # APPEND
        print(f"  DEBUG: APPEND: {new_state}")
        heapq.heappush(state_queue, new_state)
    elif new_state.cost < _exist_state.cost:
        # REPLACE
        print(f"  DEBUG: REPLACE: {_exist_state} --> {new_state}")
        state_queue.remove(_exist_state)
        state_queue.append(new_state)    # heapify した後に heappush しても良い
        heapq.heapify(state_queue)  # Rebalance of heap tree
    else:
        # DROP
        print(f"  DEBUG: DROP: {new_state}")
        
    print(state_queue)

#DEBUG
print('STATE_QUEUE:')
display(state_queue)


----- enqueue
  DEBUG: APPEND: State(id=0, cost=10, parent_id=-1)
[State(id=0, cost=10, parent_id=-1)]
  DEBUG: APPEND: State(id=1, cost=8, parent_id=-1)
[State(id=1, cost=8, parent_id=-1), State(id=0, cost=10, parent_id=-1)]
  DEBUG: APPEND: State(id=5, cost=12, parent_id=-1)
[State(id=1, cost=8, parent_id=-1), State(id=0, cost=10, parent_id=-1), State(id=5, cost=12, parent_id=-1)]
  DEBUG: REPLACE: State(id=5, cost=12, parent_id=-1) --> State(id=5, cost=9, parent_id=-1)
[State(id=1, cost=8, parent_id=-1), State(id=0, cost=10, parent_id=-1), State(id=5, cost=9, parent_id=-1)]
  DEBUG: DROP: State(id=1, cost=11, parent_id=-1)
[State(id=1, cost=8, parent_id=-1), State(id=0, cost=10, parent_id=-1), State(id=5, cost=9, parent_id=-1)]
STATE_QUEUE:


[State(id=1, cost=8, parent_id=-1),
 State(id=0, cost=10, parent_id=-1),
 State(id=5, cost=9, parent_id=-1)]

In [46]:
# ---------------------------------------------------------
# dequeue
print('\n----- dequeue (1)')
a = heapq.heappop(state_queue)
print(a)

print('\n----- dequeue (2)')
a = heapq.heappop(state_queue)
print(a)

# DEBUG
print('STATE_QUEUE:')
display(state_queue)


----- dequeue (1)
State(id=1, cost=8, parent_id=-1)

----- dequeue (2)
State(id=5, cost=9, parent_id=-1)
STATE_QUEUE:


[State(id=0, cost=10, parent_id=-1)]

## まとめ

本資料は以上です．

(tut3-appendix)=

## Appendix

ここから下はおまけです．
一通り課題を終えた後に，時間があれば確認してください．より深い考察ができるようになるかもしれません．

なお，ここに書かれた内容や数値をそのまま書きうつしてもレポートにはなりません．

(tut3-appendix2)=

### Appendix2: ヒープ構造は必要か？ &mdash; 実装方針についてのちょっとした考察

せっかくなので，「[Tutorial 2 の Appendix 1](Tutorial_2-data-structure.ipynb#tut2-appendix1)」と同じように，
簡単に速度も確認しておきましょう．

ヒープ構造は，アルゴリズムとデータ構造の講義でも難関の一つかなと思います．  
初見では，やたらとややこしい処理をしてるな・・・と思いませんでしたか？  
そしてその思いから『そんなややこしいものは使いたくない』という思いにつながってしまった人もいるでしょう．

実際のところ，普通の Queue の実装の中でも enqueue のたびにソートしておけば，「最小値を $O(1)$ で取り出す」という機能は実現できそうです．  
ソートさえ知っていれば誰でも書けますし，同レベルの同僚や後輩らにも意図が伝わりやすいでしょうから，いいことづくめのようにも見えます．

さて，どれぐらい違うのでしょうか？  
本節に取り組んでみれば，特定の処理をする際にヒープを使いたくなる・・・はず（？）

試しに，5回enqueue，3回dequeue を 100 回ぐらい繰り返して計測してみましょう（最終的には200個のデータが残ります）．

`%%timeit` を使うと，セル全体を何回も繰り返して，平均速度を計測してくれます．結果の単位を読み間違えないように．

In [47]:
%%timeit
# Stupid Priority Queue
state_queue = deque()

for _ in range(100):
    ## ENQUEUE 5 elements
    for n in [state1, state2, state3, state4, state5]:
        state_queue.append(n)
        state_queue = deque(sorted(state_queue))  # <-- NEW 挿入後にソートする

    ## DEQUEUE 3 elements
    a = state_queue.popleft()
    a = state_queue.popleft()
    a = state_queue.popleft()

2.82 ms ± 75.2 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [48]:
%%timeit
# Priority Queue (heap)
state_queue = list()

for _ in range(100):
    ## ENQUEUE 5 elements
    for n in [state1, state2, state3, state4, state5]:
        heapq.heappush(state_queue, n)

    ## DEQUEUE 3 elements
    a = heapq.heappop(state_queue)
    a = heapq.heappop(state_queue)
    a = heapq.heappop(state_queue)

249 μs ± 1.19 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [49]:
%%timeit
# Unique Priority Queue (heap)
state_queue = list()

for _ in range(100):
    ## ENQUEUE 5 elements
    for new_state in [state1, state2, state3, state4, state5]:
        _exist_state = [x for x in state_queue if new_state.id == x.id]
        _exist_state = _exist_state[0] if _exist_state else None
        if _exist_state is None:
            heapq.heappush(state_queue, new_state)
        elif new_state.cost < _exist_state.cost:
            state_queue.remove(_exist_state)
            heapq.heapify(state_queue)  # Rebalance the heap tree
            heapq.heappush(state_queue, new_state)

    ## DEQUEUE 3 elements
    a = heapq.heappop(state_queue)
    a = heapq.heappop(state_queue)
    a = heapq.heappop(state_queue)

122 μs ± 322 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


```{exercise}
※これは Appendix 内の exercise です．

まずは，上記の例に基づいて，計算量を記録してください．  
3アルゴリズムの間での比較と考察もしてみましょう．
考察では，なぜそのような結果になったのかの説明もしてくださいね．

次に，上記の計測例を書き換えて，popする回数を3回ではなく1回にして，
3アルゴリズムの計算時間を改めて比較と考察をしてみてください．
(`Ctrl + /` で，DEQUEUE処理のうち2行をコメントトグルすると簡単ですよ．)

最後に，pop回数を減らしたときに，各アルゴリズムの中で計算時間が増えたか減ったかについて，確認してください．  
各アルゴリズムで変化の傾向が異なる(はず)です．この理由を説明できますか？

:::{hint}
オーダーを議論する際の $N$ に注意しましょう．

今回の例では，ループの何巡目か，によって $N$ が常に変動します．  
ちなみに，次回以降の画像実験では，処理中の $N$ は固定とみなせるケースが多いはずです．

ここで考えたことは，ストリーム処理([wikipedia](https://ja.wikipedia.org/wiki/%E3%82%B9%E3%83%88%E3%83%AA%E3%83%BC%E3%83%A0_(%E3%83%97%E3%83%AD%E3%82%B0%E3%83%A9%E3%83%9F%E3%83%B3%E3%82%B0)))
の特徴として覚えておくと，どこかで役に立つでしょう．

例えば，音声処理実験では，時々刻々と音声データが取得できるストリーミング処理を扱うことがあります．  
また，情報工学実験Cのネットワーク実験でも，インターネットの接続先ホストから時々刻々とデータ（パケット）が送られてくるストリーム処理を扱う（はず）です．
:::
```